In [1]:
import duckdb

base_url = "https://saisonmanager.de"
api_url = base_url + "/api/v2"
leagues_url = api_url + "/leagues.json"
standings_url = api_url + "/leagues/{league_id}/table.json"
scorers_url = api_url + "/leagues/{league_id}/scorer.json"
schedule_url = api_url + "/leagues/{league_id}/schedule.json"
game_url = api_url + "/games/{game_id}.json"

conn = duckdb.connect("floorball-verbandsliga-ost.db")

In [2]:
league_id = 1397

In [4]:
conn.sql(
    f"""
CREATE OR REPLACE TABLE schedule AS SELECT * FROM '{schedule_url.format(league_id=league_id)}';
CREATE OR REPLACE TABLE standings AS SELECT * FROM '{standings_url.format(league_id=league_id)}';
CREATE OR REPLACE TABLE scorers AS SELECT * FROM '{scorers_url.format(league_id=league_id)}';
"""
)

game_urls = [
    game_url.format(game_id=game_id)
    for game_id in conn.sql("SELECT game_id FROM schedule;").df()["game_id"]
]
conn.sql(
    f"""
CREATE OR REPLACE TABLE games AS
    SELECT * FROM read_json_auto([{','.join([f"'{url}'" for url in game_urls])}], union_by_name = true);
"""
)

In [5]:
conn.sql(
    """
CREATE OR REPLACE TABLE teams AS
SELECT
    team_id AS id,
    team_name AS name,
FROM standings;
SELECT * FROM teams;
"""
)

┌───────┬────────────────────────────┐
│  id   │            name            │
│ int64 │          varchar           │
├───────┼────────────────────────────┤
│  5807 │ Floorball Tigers Magdeburg │
│  5811 │ UV Zwigge 07               │
│  5806 │ USV Jena                   │
│  5805 │ USV Halle Saalebiber II    │
│  5804 │ UHC Elster                 │
│  5808 │ FBC Phönix Leipzig         │
│  5809 │ USV TU Dresden II          │
│  5810 │ Red Devils Wernigerode II  │
└───────┴────────────────────────────┘

In [6]:
conn.sql(
    """
CREATE OR REPLACE TABLE events AS SELECT
    game_id,
    event.event_id AS id,
    event.event_type AS type,
    CASE WHEN event.event_team = 'home' THEN home_team_id ELSE guest_team_id END AS team_id,
    event.period AS period,
    event.home_goals AS home_goals,
    event.guest_goals AS guest_goals,
    make_time(
        0,
        extract('minute' FROM try_strptime(event.time, '%M:%S')),
        extract('second' FROM try_strptime(event.time, '%M:%S'))
    ) AS time,
    event.penalty_type AS penalty_type,
    event.penalty_reason AS penalty_reason,
    event.penalty_reason_string AS penalty_reason_name,
    CASE WHEN event.event_team = 'home' THEN home_players[event.number][1].player_id ELSE guest_players[event.number][1].player_id END AS player_id,
    CASE WHEN event.event_team = 'home' THEN home_players[event.assist][1].player_id ELSE guest_players[event.assist][1].player_id END AS assist_player_id,
FROM (
    SELECT
        id AS game_id,
        home_team_id,
        guest_team_id,
        map_from_entries(
            list_transform(players.home, player -> (player.trikot_number, player))
        ) AS home_players,
        map_from_entries(
            list_transform(players.guest, player -> (player.trikot_number, player))
        ) AS guest_players,
        unnest(events) AS event,
    FROM games
    WHERE ended
);
DESCRIBE events;
"""
)

┌─────────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│     column_name     │ column_type │  null   │   key   │ default │ extra │
│       varchar       │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ game_id             │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ id                  │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ type                │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ team_id             │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ period              │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ home_goals          │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ guest_goals         │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ time                │ TIME        │ YES     │ NULL    │ NULL    │  NULL │
│ penalty_type        │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ penalty_re

In [7]:
conn.sql(
    """
CREATE OR REPLACE TABLE players AS SELECT DISTINCT ON (id)
    player.player_id AS id,
    team_id,
    player.player_name AS lastname,
    player.player_firstname AS firstname,
FROM (
    SELECT
        unnest(players.home) AS player,
        home_team_id AS team_id
    FROM games
    UNION
    SELECT
        unnest(players.guest) AS player,
        guest_team_id AS team_id
    FROM games
);
DESCRIBE players;
"""
)

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ id          │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ team_id     │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ lastname    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ firstname   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

In [15]:
conn.sql(
    """
SELECT player.lastname, player.firstname, event.penalty_reason_name
FROM events event JOIN players player ON (event.player_id = player.id)
WHERE type = 'penalty'
"""
)

┌─────────────┬───────────────┬──────────────────────────┐
│  lastname   │   firstname   │   penalty_reason_name    │
│   varchar   │    varchar    │         varchar          │
├─────────────┼───────────────┼──────────────────────────┤
│ Starystach  │ Justus        │ Stockschlag              │
│ Kraft       │ Marco         │ Stockschlag              │
│ Winkelbauer │ Fabian        │ Reklamieren              │
│ Steinert    │ Till          │ Stockschlag              │
│ Knauer      │ Max           │ Stockschlag              │
│ Melzian     │ Virginia      │ Stockschlag              │
│ Schulze     │ Christian     │ Stockschlag              │
│ Zeiske      │ Leo Alexander │ Stockschlag              │
│ Knauer      │ Max           │ Stoßen                   │
│ Zoberbier   │ Jens          │ Stoßen                   │
│   ·         │  ·            │   ·                      │
│   ·         │  ·            │   ·                      │
│   ·         │  ·            │   ·                     